
Improve tongue tracking by removing connected components

Tongue tracking produces a tongue mask which may contain artifact. The largest connected component should be the tongue, therefore other connected components can be removed.

Data format: takes advantage of sparseness
Frames:
Rows are numbered 1, 2,... however many frames from recording are blank. Blank frames are removed but rows remain numbered 1, 2,... so the actual frame number is recorded per row

Height, widths, probs:
Many pixels in the image are not part of the tongue. Pixels are represented as 3-tuples of (x, y, prob) where prob is the neural network prediction of the pixel being a tongue pixel- prob is generally high since only pixels with a high probability were kept. 

In [130]:
import h5py
tongue_filepath = "data/phox2b38_20240307_1_tongue.h5"

f = h5py.File(tongue_filepath, "r")
frames = f.get("frames")
heights = f.get("heights")
widths = f.get("widths")
probs = f.get("probs")
len(frames)


64645

In [4]:
import numpy as np
import cv2

def process_img(img):
	# identify 2 largest CCs (background and tongue)
	nb_components, labels, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
	sizes = sorted([(s, i) for i, s in enumerate(stats[:, -1])], reverse=True)

	# color anyting not bg/tongue as black
	for i in range(img.shape[0]):
		img[i][np.logical_and(labels[i] != sizes[0][1], labels[i] != sizes[1][1])] = [0]

	return img

def plot_frame(heights, widths, img_height, img_width, name):
	img = np.zeros((img_height, img_width, 1), dtype=np.uint8)

	# plotting pixels
	for i in range(len(heights)):
		img[heights[i]][widths[i]] = [255] 

	# write unchanged masks
	cv2.imwrite(f"output/raw{name}.png", img)

	# write processed masks
	img = process_img(img)
	cv2.imwrite(f"output/{name}.png", img)

asdf


In [141]:
for i in range(100, 200):
	plot_frame(heights[i], widths[i], 256, 256, f"{frames[i]}")